# Notebook #1: Captura de datos y limpieza

In [54]:
# Librerías para tratamiento de datos

import pandas as pd
import geopandas as gpd
pd.set_option('display.max_columns', None) # Parámetro que modifica la visualización de los DFs
import numpy as np
import re

# Librería para el acceso a variables y funciones
import sys
sys.path.append("../")
from src import soporte_funciones as sf #Archivo .py donde encontraremos todas nuestras funciones
#from src import soporte_variables as sv

# Librería para acceder a funcionalidades del sistema operativo
import os

# Librerías para trabajar con distintos formatos de archivos
import pickle
import json

# Librería para ignorar avisos
import warnings
warnings.filterwarnings("ignore") # Ignora TODOS los avisos

## API AirBnB

In [ ]:
#resultados_airbnb = sf.consulta_airbnbs("Madrid", "2025-01-31", "2025-02-02", 11)

100%|██████████| 11/11 [03:56<00:00, 21.52s/it]


In [ ]:
#len(resultados_airbnb)

11

In [ ]:
#with open("../datos/origen/airbnb.json", "w") as json_file:
#    json.dump(resultados_airbnb, json_file, indent=4)

In [4]:
with open("../datos/origen/airbnb.json", 'r') as file:
    resultados_airbnb = json.load(file)

In [9]:
df_airbnb = sf.dataframe_airbnb(resultados_airbnb)

In [10]:
df_airbnb.shape

(360, 4)

- Usaremos la función `traducir_es`, que recibe un texto y lo traduce a español haciendo uso de Google Translate, y la aplicaremos a la columna "Descripción", para traducir su contenido a español.

In [11]:
df_airbnb["Descripcion"] = df_airbnb["Descripcion"].apply(sf.traducir_es)

In [12]:
df_airbnb.sample(3)

,Latitud,Longitud,Descripcion,Precio Total
123,40.422270,-3.701760,Habitación grande privada con 3 balcones.Gran vía,204
232,40.429271,-3.695078,Loft de lujo en la calle Almagro,339
308,40.420123,-3.698174,Madrid flats Chueca,397


- Convertimos la latitud y la longitud a geopuntos, cambiamos el formato de CRS (Coordinate Reference System) para que estandarizar el formato con los datos obtenidos de los municipios de Madrid, y guardamos en un archivo de tipo `geojason`.

In [ ]:
gdf_airbnb = gpd.GeoDataFrame(df_airbnb, geometry=gpd.points_from_xy(df_airbnb.Longitud, df_airbnb.Latitud))
gdf_airbnb.crs = "EPSG:4326"
#gdf_airbnb.to_file('../datos/origen/airbnb.geojson', driver='GeoJSON')

In [14]:
gdf_distritos = gpd.read_file("../datos/origen/madrid-districts.geojson")

In [15]:
gdf_distritos = gdf_distritos.rename(columns={"name":"Distrito","cartodb_id":"ID_Distrito"})
gdf_distritos.drop(columns= ["created_at", "updated_at"], inplace=True)

In [16]:
gdf_sjoin = gpd.sjoin(gdf_airbnb, gdf_distritos, how="inner", predicate="within")
gdf_sjoin = gdf_sjoin.drop(columns="index_right")

In [17]:
gdf_sjoin.head(1)

,Latitud,Longitud,Descripcion,Precio Total,geometry,Distrito,ID_Distrito
0,40.411549,-3.697992,Acogedor baño de doble baño en el corazón de M...,186,POINT (-3.69799 40.41155),Centro,1


In [47]:
gdf_sjoin_final = gdf_sjoin[["ID_Distrito", "Precio Total", "Descripcion", "geometry"]]

In [48]:
gdf_sjoin_final.sample(2)

,ID_Distrito,Precio Total,Descripcion,geometry
54,15,82,Habitación acogedora a los 15-20 min.desde el ...,POINT (-3.64209 40.43146)
243,1,385,Aparte.Sol central,POINT (-3.70231 40.41898)


In [169]:
gdf_sjoin_final.dtypes

ID_Distrito        int32
Precio Total       int64
Descripcion       object
geometry        geometry
dtype: object

In [ ]:
#gdf_sjoin_final.to_file('../datos/finales/airbnb.geojson', driver='GeoJSON')
#gdf_sjoin.to_file('../datos/finales/airbnb.shp')

## Creación del DF Distritos
- En él se listan los distritos de Madrid junto con los códigos asignados por el ayuntamiento.
- Guardamos en .CSV.

In [33]:
df_distritos = gdf_distritos[["ID_Distrito", "Distrito", "geometry"]]
#df_distritos.to_file('../datos/finales/distritos.geojson', driver='GeoJSON')
df_distritos

,ID_Distrito,Distrito,geometry
0,1,Centro,"MULTIPOLYGON (((-3.69185 40.40853, -3.69189 40..."
1,2,Arganzuela,"MULTIPOLYGON (((-3.70258 40.40638, -3.70166 40..."
2,3,Retiro,"MULTIPOLYGON (((-3.66279 40.4097, -3.66384 40...."
3,4,Salamanca,"MULTIPOLYGON (((-3.65809 40.43945, -3.65828 40..."
4,5,Chamartin,"MULTIPOLYGON (((-3.67231 40.48388, -3.67237 40..."
5,6,Tetuan,"MULTIPOLYGON (((-3.69633 40.47572, -3.69619 40..."
6,7,Chamberi,"MULTIPOLYGON (((-3.68991 40.44737, -3.69048 40..."
7,8,Fuencarral-El Pardo,"MULTIPOLYGON (((-3.64131 40.63922, -3.64118 40..."
8,9,Moncloa-Aravaca,"MULTIPOLYGON (((-3.79973 40.47063, -3.79887 40..."
9,10,Latina,"MULTIPOLYGON (((-3.7213 40.41256, -3.72051 40...."


In [168]:
df_distritos.dtypes

ID_Distrito       int32
Distrito         object
geometry       geometry
dtype: object

## WebScraping de Redpiso

In [ ]:
#sopas_redpiso = sf.scraping_alquileres_redpiso(50)

In [ ]:
#with open('../datos/origen/sopas_redpiso.pkl', 'wb') as file:
#    pickle.dump(sopas_redpiso, file)

In [21]:
with open('../datos/origen/sopas_redpiso.pkl', 'rb') as file:
    sopas_redpiso = pickle.load(file)

In [22]:
df_redpiso = sf.dataframe_redpiso(sopas_redpiso)

- Vemos que, hemos obtenido un DF con un total de 599 viviendas en alquiler.

In [23]:
df_redpiso.shape

(600, 2)

- Procederemos ahora a limpiar los datos:
1. Aplicaremos la función `extraer_distrito` a la columna descripción. Esta función aplica un patrón de Regex al string para obtener únicamente el distrito. Si no lo encuentra, devuelve "Distrito no identificado". Limpiaremos esos registros posteriormente.
2. En la columna precio sustituimos los puntos, los signos de euro y los strings "a consultar".
3. Homogenizamos los nombres de los distritos eliminando las tildes y los nombres generales. 

In [24]:
df_redpiso["Distrito"] = df_redpiso['Descripcion'].apply(sf.extraer_distrito)
df_redpiso["Descripcion"] = df_redpiso["Descripcion"].str.title()
df_redpiso["Precio"] = df_redpiso["Precio"].str.replace("A consultar","0")
df_redpiso["Precio"] = df_redpiso["Precio"].str.replace(".","")
df_redpiso["Precio"] = df_redpiso["Precio"].str.replace(" €","").astype(int)
df_redpiso["Distrito"] = df_redpiso["Distrito"].str.replace("Villa de Vallecas-Ensanche y Santa Eugenia","Puente de Vallecas")
df_redpiso["Distrito"] = df_redpiso["Distrito"].str.replace("San Blas-Canillejas","San Blas")
df_redpiso["Distrito"] = df_redpiso["Distrito"].str.replace("Vicálvaro-Ambroz-Centro-Valdebernardo-Valderribas","Vicálvaro")
df_redpiso["Distrito"] = df_redpiso["Distrito"].str.replace("Chamberí","Chamberi")
df_redpiso["Distrito"] = df_redpiso["Distrito"].str.replace("Chamartín","Chamartin")
df_redpiso["Distrito"] = df_redpiso["Distrito"].str.replace("Vicálvaro","Vicalvaro")
df_redpiso["Distrito"] = df_redpiso["Distrito"].str.replace("Tetuán","Tetuan")

- Tras aplicar el patrón de Regex, nos hemos quedado con 20 entradas sin identificar. Asignaremos esos distritos manualmente y comprobaremos que se hayan eliminado.

In [25]:
df_redpiso["Distrito"][0] = "Villaverde"
df_redpiso["Distrito"][13] = "Moncloa-Aravaca"
df_redpiso["Distrito"][39] = "Moncloa-Aravaca"
df_redpiso["Distrito"][112] = "Moncloa-Aravaca"
df_redpiso["Distrito"][187] = "Villaverde"
df_redpiso["Distrito"][235] = "Centro"
df_redpiso["Distrito"][257] = "Puente de Vallecas"
df_redpiso["Distrito"][273] = "Centro"
df_redpiso["Distrito"][288] = "Moncloa-Aravaca"
df_redpiso["Distrito"][380] = "Usera"
df_redpiso["Distrito"][387] = "Moncloa-Aravaca"
df_redpiso["Distrito"][446] = "Centro"
df_redpiso["Distrito"][464] = "Moncloa-Aravaca"
df_redpiso["Distrito"][493] = "Centro"
df_redpiso["Distrito"][501] = "Centro"
df_redpiso["Distrito"][527] = "Chamberi"
df_redpiso["Distrito"][551] = "Chamartin"
df_redpiso["Distrito"][568] = "Moncloa-Aravaca"
df_redpiso["Distrito"][591] = "Moncloa-Aravaca"
df_redpiso["Distrito"][592] = "Moncloa-Aravaca"

In [26]:
df_redpiso[df_redpiso["Distrito"].str.contains("Distrito no identificado", case=False, na=False)]

,Descripcion,Precio,Distrito


- Comprobamos nuevamente que los nombres de los distritos son consistentes y no tenemos registros inesperados.

In [27]:
df_redpiso["Distrito"].unique()

array(['Villaverde', 'Chamberi', 'Salamanca', 'Ciudad Lineal',
       'Puente de Vallecas', 'Barajas', 'San Blas', 'Arganzuela',
       'Tetuan', 'Moncloa-Aravaca', 'Latina', 'Hortaleza',
       'Fuencarral-El Pardo', 'Retiro', 'Centro', 'Carabanchel',
       'Chamartin', 'Moratalaz', 'Usera', 'Vicalvaro'], dtype=object)

- Eliminamos los registros donde el precio del alquiler sea cero, debido a que distorsionarían el análisis.

In [28]:
df_redpiso["Precio"].value_counts()

Precio
1100    53
1200    51
0       43
850     42
1000    40
        ..
2350     1
550      1
690      1
720      1
1480     1
Name: count, Length: 83, dtype: int64

In [29]:
df_redpiso = df_redpiso[
    (df_redpiso["Precio"] != 0)
]

- Tras eliminar los registros, nos quedamos con 556 pisos repartidos en 20 distritos, de un total de 21 que tenemos en Madrid.

In [30]:
df_redpiso["Distrito"].value_counts()

Distrito
Chamberi               74
Chamartin              69
Centro                 62
Fuencarral-El Pardo    47
Hortaleza              44
Puente de Vallecas     43
Arganzuela             35
Salamanca              33
Latina                 31
Moncloa-Aravaca        26
Tetuan                 25
Retiro                 17
Carabanchel            15
San Blas               11
Villaverde              8
Ciudad Lineal           6
Vicalvaro               6
Barajas                 2
Moratalaz               2
Usera                   1
Name: count, dtype: int64

- Para asignar el ID a cada municipio, realiaremos un merge con el dataframe de distritos. Reordenaremos las columnas y lo guardaremos en un archivo CSV.

In [34]:
df_redpiso_merge = df_redpiso.merge(df_distritos, how="inner", left_on="Distrito", right_on="Distrito")

In [35]:
df_redpiso_merge.drop(columns = ["Distrito", "geometry"], inplace=True)
df_redpiso_merge = df_redpiso_merge[["ID_Distrito", "Precio", "Descripcion"]]
df_redpiso_merge.head(5)

,ID_Distrito,Precio,Descripcion
0,17,850,"Piso En Alquiler En Villaverde, Madrid, Madrid"
1,7,1100,"Piso En Alquiler En Calle Cristobal Bordiu, Rí..."
2,4,2000,"Apartamento En Alquiler En Calle Fundadores, F..."
3,15,744,"Piso En Alquiler En Calle Pepe Isbert, Pueblo ..."
4,13,750,Estudio En Alquiler En Calle Embalse De Navace...


In [165]:
df_redpiso_merge.dtypes

ID_Distrito     int32
Precio          int64
Descripcion    object
dtype: object

In [ ]:
#df_redpiso_merge.to_csv("../datos/finales/redpiso.csv")

## API Idealista

In [ ]:
#resultados_idealista = sf.consulta_idealista("0-EU-ES-28-07-001-079", "Madrid", 10)

In [ ]:
#with open("../datos/origen/idealista.json", "w") as json_file:
#    json.dump(resultados_idealista, json_file, indent=4)

In [37]:
with open("../datos/origen/idealista.json", 'r') as file:
    resultados_idealista = json.load(file)

In [38]:
df_idealista = sf.dataframe_idealista(resultados_idealista)

In [39]:
df_idealista["Tipo"] = df_idealista["Tipo"].apply(sf.traducir_es)
df_idealista["Direccion"] = df_idealista["Direccion"].str.title()
df_idealista["Descripcion"] = df_idealista["Descripcion"].str.title()

In [ ]:
df_idealista.head(2)

,Latitud,Longitud,Precio,Tipo,Planta,Tamaño,Habitaciones,Baños,Dirección,Descripción
0,40.432201,-3.714278,1350.0,departamento,1,73.0,2,1,"Calle De Andres Mellado, 18",Piso Con Una Habitación Con Cama De Matrimonio...
1,40.414669,-3.705414,2300.0,departamento,5,95.0,2,2,Calle Del Marqués Viudo De Pontejos,S C A L A | Properties Presenta Ático Amuebl...


- Cuando un valor se repite muchas veces, como puede ser el caso del tipo de vivienda, para crear nuestra base de datos, lo correcto sería crear una nueva tabla con identificadores únicos para cada tipo de vivienda, así, normalizamos los datos, eliminando redundancias. Por una limitación de tiempo, no se hará en este proyecto.

In [52]:
df_idealista["Tipo"].value_counts()

Tipo
departamento    321
estudio          37
ático            25
dúplex           11
chalet            6
Name: count, dtype: int64

In [ ]:
gdf_idealista = gpd.GeoDataFrame(df_idealista, geometry=gpd.points_from_xy(df_idealista.Longitud, df_idealista.Latitud))
gdf_idealista.crs = "EPSG:4326"
#gdf_idealista.to_file('../datos/origen/idealista.geojson', driver='GeoJSON')

In [41]:
gdf_sjoin2 = gpd.sjoin(gdf_idealista, gdf_distritos, how="inner", predicate="within")
gdf_sjoin2 = gdf_sjoin2.drop(columns="index_right")


In [163]:
gdf_sjoin2_final = gdf_sjoin2[['ID_Distrito', 'Precio', 'Tipo', 'Planta', 'Tamanio',
       'Habitaciones', 'Banios', 'Direccion', 'Descripcion', 'geometry']]
gdf_sjoin2_final["Tamanio"] = gdf_sjoin2_final["Tamanio"].astype(int)

In [45]:
gdf_sjoin2_final.sample(2)

,ID_Distrito,Precio,Tipo,Planta,Tamanio,Habitaciones,Banios,Direccion,Descripcion,geometry
281,3,1600.0,departamento,bj,56.0,1,1,Avenida De Menéndez Pelayo,Urbanissimo Real Estate Comercializa Apartamen...,POINT (-3.67598 40.41235)
372,15,1700.0,departamento,bj,90.0,3,1,Calle Del Buen Gobernador,Se Alquila Luminoso Piso En El Céntrico Barrio...,POINT (-3.65578 40.4334)


In [164]:
gdf_sjoin2_final.dtypes

ID_Distrito        int32
Precio           float64
Tipo              object
Planta            object
Tamanio            int64
Habitaciones       int64
Banios             int64
Direccion         object
Descripcion       object
geometry        geometry
dtype: object

In [ ]:
#gdf_sjoin2_final.to_file('../datos/finales/idealista.geojson', driver='GeoJSON')

## WebScraping Ayuntamiento de Madrid

- Descargamos CSV

In [ ]:
#sf.scraping_ayuntamiento()

Cookies aceptadas
Click en todos los distritos
Click en totales barrios
Click en todos los períodos
Click en todas las medidas
Click en todas las nacionalidades
Click en generar CSV


In [59]:
os.listdir("../datos/origen/")

['airbnb.json',
 '.DS_Store',
 '31097.csv',
 'Datos de la serie 0307010000022.csv',
 'madrid-districts.geojson',
 'sopas_redpiso.pkl',
 'airbnb.geojson',
 'idealista.geojson',
 'idealista.json']

In [60]:
os.rename("../datos/origen/Datos de la serie 0307010000022.csv", "../datos/origen/extranjeros_madrid.csv")

In [130]:
df_extranjeros = pd.read_csv("../datos/origen/extranjeros_madrid.csv", sep=",", encoding="latin-1")
encabezados_extranjeros = df_extranjeros.columns

In [131]:
nuevos_encabezados_ext = [re.findall(r"\d+", enc)[0] if re.findall(r"\d+", enc) else enc for enc in encabezados_extranjeros]
nuevos_encabezados_ext[0] = "Periodo"
nuevos_encabezados_ext[1] = "Categoria"
nuevos_encabezados_ext2 = [enc.lstrip("0") for enc in nuevos_encabezados_ext]
df_extranjeros.columns = nuevos_encabezados_ext2
df_extranjeros["Categoria"] = df_extranjeros["Categoria"].str.replace("Espaola","Espaniola")
df_extranjeros

,Periodo,Categoria,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
0,2018,Total,359,1010,505,1443,1528,2248,899,4327,913,1966,4232,2223,2926,26,1967,3455,2934,3295,917,1853,919
1,2018,Espaniola,-631,706,140,361,1085,744,438,2804,740,-506,697,-36,-184,-367,190,2332,575,2368,562,209,725
2,2018,Extranjera,991,305,366,1082,442,1501,461,1523,177,2469,3533,2257,3110,393,1774,1125,2360,925,354,1644,194
3,2019,Total,2533,917,338,886,887,2024,1072,3088,1566,2449,4856,2497,4243,297,1761,4360,3284,2783,1174,1967,1151
4,2019,Espaniola,806,171,-523,-308,247,119,71,1456,704,-675,20,-118,-415,-616,-670,2346,527,1355,556,-151,760
5,2019,Extranjera,1728,744,861,1194,641,1906,1000,1632,859,3125,4836,2616,4658,914,2435,2014,2755,1423,618,2117,392
6,2020,Total,5677,1912,1186,1827,1808,3497,1483,3996,2375,4022,7321,3456,6248,1089,3737,5139,5638,4189,1941,3070,1088
7,2020,Espaniola,2362,824,209,-229,727,1050,169,1935,1187,-230,1117,53,574,-171,561,3001,1797,2220,1046,282,649
8,2020,Extranjera,3314,1088,977,2056,1081,2448,1314,2061,1189,4250,6203,3403,5671,1260,3176,2138,3842,1966,892,2785,439
9,2021,Total,622,-1877,-2284,-2217,-2185,-2101,-2743,-2921,-1161,-2632,-2297,-844,-2578,-2048,-3620,-609,-25,-77,1399,-1379,-71


In [126]:
#Convertir a formato largo
df_ext_melted = pd.melt(df_extranjeros, id_vars=["Periodo", "Categoria"], var_name="ID_Distrito", value_name="Habitantes")
df_ext_melted

,Periodo,Categoria,ID_Distrito,Habitantes
0,2018,Total,1,359
1,2018,Espaniola,1,-631
2,2018,Extranjera,1,991
3,2019,Total,1,2533
4,2019,Espaniola,1,806
...,...,...,...,...
310,2021,Espaniola,21,-152
311,2021,Extranjera,21,78
312,2022,Total,21,129
313,2022,Espaniola,21,214


In [155]:
# Pivotar para obtener una fila por distrito y por año
df_ext_reshaped = df_ext_melted.pivot_table(index=["ID_Distrito", "Periodo"], columns="Categoria", values="Habitantes").reset_index()
df_ext_reshaped[["Espaniola","Extranjera","Total"]] = df_ext_reshaped[["Espaniola","Extranjera","Total"]].astype(int)
df_ext_reshaped

Categoria,ID_Distrito,Periodo,Espaniola,Extranjera,Total
0,1,2018,-631,991,359
1,1,2019,806,1728,2533
2,1,2020,2362,3314,5677
3,1,2021,-2015,2637,622
4,1,2022,-1947,519,-1430
...,...,...,...,...,...
100,9,2018,740,177,913
101,9,2019,704,859,1566
102,9,2020,1187,1189,2375
103,9,2021,-1441,281,-1161


In [161]:
df_ext_reshaped.dtypes

Categoria
ID_Distrito    object
Periodo         int64
Espaniola       int64
Extranjera      int64
Total           int64
dtype: object

In [ ]:
#df_ext_reshaped.to_csv("../datos/finales/extranjeros_madrid.csv")

## WebScraping INE

- Descarga csv

In [ ]:
#sf.scraping_ine()

Cookies aceptadas
Quitadas opciones por defecto
Click en todos los años
Desplegable Madrid abierto
Click en distritos
Click en descarga
Click en CSV


In [61]:
os.listdir("../datos/origen/")

['airbnb.json',
 '.DS_Store',
 '31097.csv',
 'extranjeros_madrid.csv',
 'madrid-districts.geojson',
 'sopas_redpiso.pkl',
 'airbnb.geojson',
 'idealista.geojson',
 'idealista.json']

In [62]:
os.rename("../datos/origen/31097.csv", "../datos/origen/ingresos_hogares_distrito.csv")

In [158]:
df_renta = pd.read_csv("../datos/origen/ingresos_hogares_distrito.csv", sep=";")
df_renta.drop(columns=["Municipios", "Secciones", "Indicadores de renta media y mediana"], inplace=True)
df_renta['Distritos'] = df_renta['Distritos'].str.extract(r'(\d{2})$', expand=False)
df_renta["Distritos"] = df_renta["Distritos"].apply(lambda x: x.lstrip("0") if isinstance(x, str) else x)
df_renta.rename(columns={"Distritos":"ID_Distrito"}, inplace=True)
df_renta["ID_Distrito"] = df_renta["ID_Distrito"].astype(int)
df_renta

,ID_Distrito,Periodo,Total
0,1,2022,20.587
1,1,2021,19.199
2,1,2020,18.314
3,1,2019,18.789
4,1,2018,17.932
...,...,...,...
163,21,2019,19.026
164,21,2018,18.514
165,21,2017,17.807
166,21,2016,17.641


In [159]:
df_renta.dtypes

ID_Distrito      int64
Periodo          int64
Total          float64
dtype: object

In [ ]:
#df_extranjeros.to_csv("../datos/finales/ingresos_hogares_distritos.csv")